# Cargo las librerías

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px

# Cargo la base de la ECH 2023 en csv

In [2]:
ECH2023 = pd.read_csv('Data/ECH_implantacion_2023.csv')
uruguay_map = gpd.read_file('Data/ine_depto.shp') 


# Caracterización de la variable e557 que identifica a quien informa los datos de la persona

Esta variable toma los valores 1, 2 y 3:
- 1 Sí, es la misma persona
- 2 No, responde un miembro de este hogar
- 3 No, responde un miembro calificado de otro hogar

## Frecuencia de la variable

In [3]:
tabla_porcentaje = ECH2023['e557'].value_counts(normalize=True).reset_index(name='Porcentaje') * 100
tabla_porcentaje = pd.DataFrame(tabla_porcentaje)

#agrego columna con los valores de la variable
tabla_porcentaje['e557'] = [2,1,3]
# creo columnas categoría y filtro
tabla_porcentaje['Categoría'] = 'Total'
tabla_porcentaje['Filtro'] = 'Total'

## Por sexo

In [4]:
# ahora hago frecuencia de esta variable pero por sexo
tabla_sexo = ECH2023.groupby('e26')['e557'].value_counts(normalize=True).reset_index(name='Porcentaje')
tabla_sexo['Porcentaje'] *= 100
# modifico las categorías de la variable sexo (e26)
tabla_sexo['e26'] = tabla_sexo['e26'].map({1: 'Hombre', 2: 'Mujer'})
tabla_sexo.rename(columns={'e26': 'Categoría'}, inplace=True)
# agrego una variable filtro
tabla_sexo['Filtro'] = 'Sexo'

In [5]:
#concateno las tablas
tabla_junta_e557 = pd.concat([tabla_porcentaje, tabla_sexo], axis=0)

## Por tramos de edad

In [6]:
#Creo una variable de tramos de edad
bins = [0, 13, 17, 40 ,64, 110]
labels = ['[0 a 13]', '[14 a 17]', '[18 a 40]','[41 a 64]' ,'[65 o más]']

ECH2023['tramo_edad'] = pd.cut(ECH2023['e27'], bins=bins, labels=labels, right=True)

In [7]:
# ahora hago frecuencia de esta variable pero por tramos de edad
tabla_edad = ECH2023.groupby('tramo_edad')['e557'].value_counts(normalize=True).reset_index(name='Porcentaje')
tabla_edad['Porcentaje'] *= 100
tabla_edad.rename(columns={'tramo_edad': 'Categoría'}, inplace=True)
tabla_edad['Filtro'] = 'Tramo etario'


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\1853299017.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_edad = ECH2023.groupby('tramo_edad')['e557'].value_counts(normalize=True).reset_index(name='Porcentaje')


In [8]:
#concateno a la tabla principal
tabla_junta_e557 = pd.concat([tabla_junta_e557, tabla_edad], axis=0)

## Por región

In [9]:
#creo variable mdeo_int a partir de la variable dpto
ECH2023['mdeo_int'] = np.where(ECH2023['dpto'] == 1, 'Montevideo', 'Interior')

In [10]:
# calculo frecuencia de la variable pero por region
tabla_region = ECH2023.groupby('mdeo_int')['e557'].value_counts(normalize=True).reset_index(name='Porcentaje')
tabla_region['Porcentaje'] *= 100
tabla_region.rename(columns={'mdeo_int': 'Categoría'}, inplace=True)
tabla_region['Filtro'] = 'Región'

In [11]:
#concateno a la tabla principal
tabla_junta_e557 = pd.concat([tabla_junta_e557, tabla_region], axis=0)

## Ajustes finales para esta tabla

In [12]:
# creo la variable descripción a partir de la e557
tabla_junta_e557['Descripción'] = tabla_junta_e557['e557'].apply(lambda x: 'Sí, es la misma persona' if x == 1 else ('No, responde un miembro de este hogar' if x == 2 else ('No, responde un miembro calificado de otro hogar' if x == 3 else 99)))

#redondeo la variable porcentaje
tabla_junta_e557['Porcentaje'] = tabla_junta_e557['Porcentaje'].round(1)

# Finalmente restablezco el índice del DataFrame concatenado
tabla_junta_e557 = tabla_junta_e557.reset_index(drop=True)


# Análisis

## Camino mínimo

Se crea un DataFrame con los datos que se utilizarán en el filtrado y las visualizaciones.

Las encuestas se puntúan según una selección de variables y valores que permiten acortar la realización de la ECH. Esta puntuación se clasifica en tres niveles: bajo, medio o alto.

Posteriormente, se agrupa la información por departamento y se suman los puntajes, normalizándolos entre 0 y 1. A continuación, se genera un mapa donde los colores varían de blanco a rojo: cuanto más rojo sea un departamento, menor será el recorrido promedio de las encuestas en esa zona. Aunque esto podría estar relacionado con características sociodemográficas propias de cada departamento, se propone comparar estos resultados aplicando un filtro adicional: la "Identificación del Informante". De esta manera, se podrá evaluar si el recorrido de la encuesta varía significativamente cuando el informante es la misma persona u otra persona informa por el. Se plantea la hipótesis de que las encuestas realizadas por informantes "proxy" tendrán un recorrido menor.

In [13]:
# Crear un DataFrame con Variables Seleccionadas ---
# Selecciono variables de interés, incluyendo 'e557' para futuros filtrados
df_selected = ECH2023[['nom_dpto', 'ESTRED13', 'c6_1', 'e38', 'e46_cv', 'e582', 'f75', 'f285', 'f288', 'f99', 'f101', 'f102', 'f299', 'f110', 'f116', 'g132', 'g140', 'h161', 'e557']]

# juntamos las categorias 2 y 3 de la variable e557, para diferenciar entre los que responden por si mismos y los que no
df_selected['Informante'] = df_selected['e557'].apply(lambda x: 1 if x == 1 else 2)
# 1 si informa por si mismo
# 2 si informa otra persona

# Implemento  un Sistema de Puntuación Gradual del Indicador
# Puntuación baja: Observaciones que cumplen solo el primer grupo de condicionesç
df_selected['low_score'] = ((df_selected['c6_1'] == 1) & (df_selected['e38'] == 1)).astype(int)

# Puntuación media: Observaciones que cumplen con más condiciones
df_selected['medium_score'] = ((df_selected['e46_cv'] == 2) | (df_selected['e582'] == 2) | (df_selected['f75'] == 1)).astype(int)

# Puntuación alta: Observaciones que cumplen con la mayoría de las condiciones
df_selected['high_score'] = (
    (df_selected['f285'] == 2) | (df_selected['f288'].isin([2, 3])) | (df_selected['f99'] == 2) |
    (df_selected['f101'].isin([2, 3, 4])) | (df_selected['f102'] == 2) | (df_selected['f299'] == 2) |
    (df_selected['f110'] == 7) | (df_selected['f116'] == 2) | (df_selected['g132'] == 3) |
    (df_selected['g140'] == 3) | (df_selected['h161'] == 2)
).astype(int)

# combino puntajes en un indicador gradual
df_selected['gradual_indicator'] = df_selected['low_score'] + df_selected['medium_score'] + df_selected['high_score']

# Agrego puntajes por departamento y estrato
df_grouped = df_selected.groupby(['nom_dpto', 'ESTRED13']).agg(
    total_score=('gradual_indicator', 'sum'),  
    count=('gradual_indicator', 'size')  
).reset_index()

# Normalizar la Puntuación a un Rango de 0 a 1 ---
# Calcula la puntuación máxima posible para la normalización
max_score = df_grouped['total_score'].max()
df_grouped['normalized_score'] = df_grouped['total_score'] / max_score  

#  Merge de datos y visualizacion de mapa
# Llevo las variables para hacer el merge a mayúscula para que se matcheen los datos correctamente
df_grouped['nom_dpto'] = df_grouped['nom_dpto'].str.upper()
uruguay_map['NOMBRE'] = uruguay_map['NOMBRE'].str.upper()

# Realizo el merge
merged_map = uruguay_map.merge(df_grouped, left_on='NOMBRE', right_on='nom_dpto', how='left')

# Ploteo el mapa
#fig, ax = plt.subplots(1, 1, figsize=(10, 10))
#merged_map.boundary.plot(ax=ax)  
#merged_map.plot(column='normalized_score', ax=ax, legend=True, cmap='Reds', missing_kwds={"color": "lightgrey"})  
#merged_map.plot(column='normalized_score', ax=ax, legend=True, cmap='Reds', missing_kwds={"color": "white"})
#plt.title('Puntaje de mínimo recorrido por departamento')
#plt.show()


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\4165347116.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Informante'] = df_selected['e557'].apply(lambda x: 1 if x == 1 else 2)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\4165347116.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['low_score'] = ((df_selected['c6_1'] == 1) & (df_selected['e38'] == 1)).astype(int)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\4165347116.py:15: SettingWithCopyWarning: 
A valu

In [44]:
# Creo una visualización Sunburst interactiva
#fig = px.sunburst(
 #   df_grouped,
  #  path=['nom_dpto', 'ESTRED13'],  
   # values='normalized_score',  # Uso puntuación normalizada para el tamaño de las secciones
    #color='normalized_score',  # Color por puntación normalizada
    #color_continuous_scale='Reds',  # Escala entre blanco y rojo
    #title="Distribución por estrato y departamento del puntaje de riesgo por menor recorrido en la encuesta"
#)

# Mejoro la visualización
#fig.update_layout(
 #   margin=dict(t=50, l=25, r=25, b=25),  # Set margins
  #  coloraxis_colorbar=dict(title="Score Normalizado")  # Title for the color bar
#)

# Show the sunburst chart
#fig.show()

## Análisis de ingresos y horas efectivas

Se realiza un análisis para poder observar si existen diferencias en los ingresos por trabajo y las horas efectivas trabajadas, entre las personas que informan por si mismas y quienes tienen un informante que responde por ellas (proxy).
Para esto se seleccionan algunas ramas de ocupación (particularmente las que tienen mayor representación en la ECH 2023), y se calcula el ingreso promedio y la media de las horas efectivas trabajadas en la semana de referencia por sexo y por nivel educativo, tanto para informantes como proxys.

In [14]:
# Definir los códigos de ocupación que nos interesan
codigos_interes = [9700, 8610, 4100, 141, 4711]

# Filtrar las filas donde f72_2 coincide con los códigos de interés
df_ocupacion_filtrada = ECH2023[ECH2023['f72_2'].isin(codigos_interes)]
df_ocupacion_filtrada['Informante'] = df_ocupacion_filtrada['e557'].apply(lambda x: 1 if x == 1 else 2)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\1221567277.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ocupacion_filtrada['Informante'] = df_ocupacion_filtrada['e557'].apply(lambda x: 1 if x == 1 else 2)


In [15]:
# Crear la variable 'NIVEDURR' siguiendo la lógica dada
df_ocupacion_filtrada['NIVEDURR'] = df_ocupacion_filtrada.apply(lambda x: (
    "9. TERCIARIA COMP" if x['e215_1'] == 1 or x['e218_1'] == 1 or x['e221_1'] == 1 or x['e224_1'] == 1 else
    "8. TERCIARIA INCOMP" if (x['e51_8'] + x['e51_9'] + x['e51_10']) > 0 else
    "7. EMS COMP" if x['e201_1c'] == 1 or x['e201_1d'] == 1 else
    "6. EMS INCOMP" if (x['e51_5'] + x['e51_6']) > 0 else
    "5. EM CICLO BASICO COMP" if x['e201_1a'] == 1 or x['e201_1b'] == 1 else
    "4. EM CICLO BASICO INCOMP" if (x['e51_4_a'] + x['e51_4_b']) > 0 else
    "3. PRIMARIA COMP" if x['e197_1'] == 1 else
    "2. PRIMARIA INCOMP" if (x['e51_2'] + x['e51_3']) > 0 else
    "0. SIN INSTRUCCION" if x['e49'] == 2 else None
), axis=1)

# Crear la variable 'NIVEDU2021'
df_ocupacion_filtrada['NIVEDU2021'] = df_ocupacion_filtrada['NIVEDURR'].apply(lambda x: (
    "0. S/INSTRUCCIÓN Y PRIM. INC" if x in ["0. SIN INSTRUCCION", "1. PRESCOLAR", "2. PRIMARIA INCOMP"] else
    "1. PRIMARIA COMPLETA O CB INCOMPLETO" if x in ["3. PRIMARIA COMP", "4. EM CICLO BASICO INCOMP"] else
    "2. CB COMPLETO O EMS INCOMP" if x in ["5. EM CICLO BASICO COMP", "6. EMS INCOMP"] else
    "3. SEC. COMPLETA O TERCIARIA INCOMP" if x in ["7. EMS COMP", "8. TERCIARIA INCOMP"] else
    "4. TERCIARIO COMPLETO O POSGRADO" if x == "9. TERCIARIA COMP" else None
))

# Crear la variable 'NIVEDU2021_2'
df_ocupacion_filtrada['NIVEDU2021_2'] = df_ocupacion_filtrada['NIVEDU2021'].apply(lambda x: (
    "1. CB INCOMPLETO O MENOS" if x in ["0. S/INSTRUCCIÓN Y PRIM. INC", "1. PRIMARIA COMPLETA O CB INCOMPLETO"] else
    "2. CB COMPLETO O EMS INCOMP" if x == "2. CB COMPLETO O EMS INCOMP" else
    "3. SEC. COMPLETA O TERCIARIA INCOMP" if x == "3. SEC. COMPLETA O TERCIARIA INCOMP" else
    "4. TERCIARIO COMPLETO O POSGRADO" if x == "4. TERCIARIO COMPLETO O POSGRADO" else None
))


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\1285675240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ocupacion_filtrada['NIVEDURR'] = df_ocupacion_filtrada.apply(lambda x: (
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\1285675240.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ocupacion_filtrada['NIVEDU2021'] = df_ocupacion_filtrada['NIVEDURR'].apply(lambda x: (
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\1285675240.py:24: SettingWithCopyWarning: 
A value is trying to

In [16]:
#creo variable horas efectivas
df_ocupacion_filtrada['Horas_efectivas'] = df_ocupacion_filtrada['f284_1']+df_ocupacion_filtrada['f284_2']+df_ocupacion_filtrada['f284_3']+df_ocupacion_filtrada['f284_4']+df_ocupacion_filtrada['f284_5']+df_ocupacion_filtrada['f284_6']+df_ocupacion_filtrada['f284_7']

tabla_estadisticas_h = (
    df_ocupacion_filtrada.groupby(['Informante','f72_2', 'NIVEDU2021_2', 'e26'])
    .agg(
        media_horas=('Horas_efectivas', 'mean'),
        media_ingreso=('PT2', 'mean')
    )
    .reset_index()
)

# Redondear las estadísticas a 1 decimal
tabla_estadisticas_h[['media_horas', 'media_ingreso']] = tabla_estadisticas_h[['media_horas', 'media_ingreso']].round(1)

# Ordenar por la columna 'Frecuencia' en orden descendente
tabla_estadisticas_h = tabla_estadisticas_h.sort_values(by='f72_2', ascending=False).reset_index(drop=True)

tabla_estadisticas_h['e26'] = tabla_estadisticas_h['e26'].apply(lambda x: '1- Hombre' if x == 1 else '2- Mujer')
 
# Modificar los valores de la variable de rama
mapeo_ciu = { 8610: 'Act. Hospitales', 141: 'Cría de ganado vacuno', 4711: 'Comercio minorista', 9700: 'Act. Hogares', 4100: 'Construcción' } 
# Aplicar el mapeo a la columna 
tabla_estadisticas_h['f72_2'] = tabla_estadisticas_h['f72_2'].map(mapeo_ciu) 




C:\Users\Usuario\AppData\Local\Temp\ipykernel_16084\473280406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ocupacion_filtrada['Horas_efectivas'] = df_ocupacion_filtrada['f284_1']+df_ocupacion_filtrada['f284_2']+df_ocupacion_filtrada['f284_3']+df_ocupacion_filtrada['f284_4']+df_ocupacion_filtrada['f284_5']+df_ocupacion_filtrada['f284_6']+df_ocupacion_filtrada['f284_7']
